# Graph Based Neutrino Reconstruction

This jupyter notebook has been created for the [Poster Contribution ID 738 on the Acat Conference 2021](https://indico.cern.ch/event/855454/contributions/4596736/).

In it, we want to take a closer look at how to setup and use a graph based neutrino reconstruction.


## Physics Primer
The central approach of the analytic neutrino reconstruction in semi-leptonic decays is to assume that the unmeasured neutrino and the measured lepton originated in a W boson decay. The mass of the W boson is well know.
With the assumption that the transverse momentum of the neutrino equals the missing transverse energy (MET) we can deduce the longitudinal momentum of the neutrino to be:

$p_{\nu,z}^{1,2} = \frac{k}{p_{l,T}^2}\left(p_{l,z} \pm E_l\,\sqrt{1-\underbrace{\left(\frac{p_{l,T}\,p_{\nu,T}}{k}\right)^2}_{\equiv\,h}}\right)$


Depending on the kinematics of an event, two cases for $h$ are possible:
* $0 \leq h \leq 1$ (real solution): In this case, the longitudinal component of the neutrino can be calculated analytically
* $h > 1$ (complex solution): In this case, the longitudinal component of the neutrino can not be calculated analytically. The usual approach is to vary the transversal component of the neutrino such, that $h = 1$. Infinitely many transverse neutrino momenta fulfill this relation. We want to take the one, that is closest to the MET. This solution is found by an event-based fit.

This calculation is exemplary for a computation in a HEP analysis which is challenging to vectorise.

First we need to import the necessary software.
Our graph library of choice will be tensorflow.

We will use:
 - `tensorflow` for building the graph
 - `numpy` to handle data outside of the graph

In [ ]:
import numpy as np
import tensorflow as tf

Now we will define some basic kinematic operations:

In [ ]:
@tf.function
def calc_vy(vx, lepx, lepy, mc, sign=+1, save=False):
    s = (mc ** 2 / 2 * (lepx ** 2 + lepy ** 2) * (2 * lepx * vx + mc ** 2 / 2)) ** 0.5
    if save:
        s = tf.where(s > 0, s, 0)
    return (lepx * lepy * vx + lepy * mc ** 2 / 2 + sign * s) / lepx ** 2


@tf.function
def shift(vx, vy, metx, mety):
    return (metx - vx) ** 2 + (mety - vy) ** 2


@tf.function
def get_k(lepx, lepy, vx, vy, mc):
    return mc ** 2 / 2 + lepx * vx + lepy * vy


@tf.function
def get_pt(px, py):
    return (px ** 2 + py ** 2) ** 0.5


@tf.function
def get_h(lepx, lepy, metx, mety, mc):
    k = get_k(lepx, lepy, metx, mety, mc)
    leppt = get_pt(lepx, lepy)
    metpt = get_pt(metx, mety)
    return (leppt * metpt / k) ** 2

Now we can define the first branch of graph, the real component of the reconstruction.

In [ ]:
# real solution
@tf.function
def neutrino_reco_real(lepe, lepx, lepy, lepz, vx, vy, mc):
    # standard reconstruction
    k = get_k(lepx, lepy, vx, vy, mc)
    leppt = get_pt(lepx, lepy)
    h = get_h(lepx, lepy, vx, vy, mc)

    # positive solution
    vzp = k / leppt ** 2 * (lepz + lepe * (1 - h) ** 0.5)
    # negative solution
    vzn = k / leppt ** 2 * (lepz - lepe * (1 - h) ** 0.5)

    vz = tf.where(tf.abs(vzp) <= tf.abs(vzn), vzp, vzn)

    return vx, vy, vz

Now we can define the second branch of the graph, the complex component of the reconstruction.
As described above, this component involves fitting the transerval component (in this case $p_x$) of the MET.

All parts which are needed for the fit (optimisers, placeholder, ...) will be part of the graph.

In [ ]:
# complex solution
@tf.function
def vx_constrain(vx, lepx, mc):
    inf = tf.constant(np.inf)
    constr_val = -(mc ** 2) / (4 * lepx)
    return tf.where(
        lepx >= 0,
        tf.clip_by_value(vx, constr_val, inf),
        tf.clip_by_value(vx, -inf, constr_val),
    )


@tf.function
def opt(vx0, lepx, lepy, metx, mety, mc, sign=+1):
    vy = calc_vy(vx0, lepx, lepy, mc, sign=sign)
    return shift(vx0, vy, metx, mety)


@tf.function
def optimize1(optimizer, vx, metx, mety, lepx, lepy, mc, sign=+1):
    # two different optimise functions to use them simultaneaously in one computing graph
    batch_size = tf.shape(metx)[0]
    for _ in range(100):
        with tf.control_dependencies(
            [
                optimizer.minimize(
                    lambda: opt(vx[:batch_size], lepx, lepy, metx, mety, mc, sign=sign),
                    var_list=[vx],
                )
            ]
        ):
            vx[:batch_size].assign(vx_constrain(vx[:batch_size], lepx, mc))
    return vx[:batch_size]


@tf.function
def optimize2(optimizer, vx, metx, mety, lepx, lepy, mc, sign=+1):
    # two different optimise functions to use them simultaneaously in one computing graph
    batch_size = tf.shape(metx)[0]
    for _ in range(100):
        with tf.control_dependencies(
            [
                optimizer.minimize(
                    lambda: opt(vx[:batch_size], lepx, lepy, metx, mety, mc, sign=sign),
                    var_list=[vx],
                )
            ]
        ):
            vx[:batch_size].assign(vx_constrain(vx[:batch_size], lepx, mc))
    return vx[:batch_size]


@tf.function
def neutrino_reco_complex(lepe, lepx, lepy, lepz, metx, mety, vx1, vx2, opt1, opt2, mc):
    # complex solution, fit using tensorflow
    vx1 = optimize1(opt1, vx1, metx, mety, lepx, lepy, mc, sign=+1)
    vx2 = optimize2(opt2, vx2, metx, mety, lepx, lepy, mc, sign=-1)

    # catch failed optimizations
    constr_val = -(mc ** 2) / (4 * lepx)
    vx1 = tf.where(tf.math.is_nan(vx1), constr_val, vx1)
    vx2 = tf.where(tf.math.is_nan(vx2), constr_val, vx2)

    # reconstruct y component
    vy1 = calc_vy(vx1, lepx, lepy, mc, sign=+1, save=True)
    vy2 = calc_vy(vx2, lepx, lepy, mc, sign=-1, save=True)

    # calculate shift between reco and met
    diff1 = shift(vx1, vy1, metx, mety)
    diff2 = shift(vx2, vy2, metx, mety)

    # take smaller shift
    vx = tf.where(diff1 <= diff2, vx1, vx2)
    vy = tf.where(diff1 <= diff2, vy1, vy2)

    # reconstruct pz component
    k = get_k(lepx, lepy, vx, vy, mc)
    leppt = get_pt(lepx, lepy)
    vz = k * lepz / leppt ** 2
    return vx, vy, vz

The flow within the graph is guided with conditional operations (`tf.where`). Here, we interleave the real component and the complex component of the reconstruction depending on the kinematics of each event.

In [ ]:
@tf.function
def neutrino_reco(lepe, lepx, lepy, lepz, metx, mety, vx1, vx2, opt1, opt2, mc):
    # include lep mass in mass constrain
    lepm = (lepe ** 2 - lepx ** 2 - lepy ** 2 - lepz ** 2) ** 0.5
    lepm = tf.where(tf.math.is_nan(lepm), tf.zeros_like(lepm), lepm)
    mc = (mc ** 2 - lepm ** 2) ** 0.5

    # interleave standard reconstruction (1) and complex solution (2)
    vx_real, vy_real, vz_real = neutrino_reco_real(lepe, lepx, lepy, lepz, metx, mety, mc)
    vx_complex, vy_complex, vz_complex = neutrino_reco_complex(
        lepe, lepx, lepy, lepz, metx, mety, vx1, vx2, opt1, opt2, mc
    )
    h = get_h(lepx, lepy, metx, mety, mc)
    vx = tf.where(h <= 1, vx_real, vx_complex)
    vy = tf.where(h <= 1, vy_real, vy_complex)
    vz = tf.where(h <= 1, vz_real, vz_complex)

    return vx, vy, vz

Finally we wrap up our graph in a `tf.keras` Model layer.
This eases the general handling (initialization, ...) of the underlying graph. 

In [ ]:
class NeutrinoReco(tf.keras.layers.Layer):
    def __init__(self, mass_constraint=80.4, optimizer="Adam", batch_size=1000):
        super(NeutrinoReco, self).__init__(name="")
        self.opt1 = tf.keras.optimizers.get(optimizer)
        self.opt2 = tf.keras.optimizers.get(optimizer)

        self.mass_constraint = mass_constraint
        self.batch_size = batch_size

    def build(self, input_shape):
        self.vx1 = self.add_weight(shape=(self.batch_size,), name="vx1")
        self.vx2 = self.add_weight(shape=(self.batch_size,), name="vx2")

    def reset_optimizer(self, *args, **kwargs):
        # reset optimizer to initial values (all zeros)
        for var in self.opt1.variables():
            var.assign(tf.zeros_like(var))
        for var in self.opt2.variables():
            var.assign(tf.zeros_like(var))

    @tf.function
    def call(self, inputs, training=False):
        self.reset_optimizer()
        lepe, lepx, lepy, lepz, metx, mety = (inputs[:, i] for i in range(6))

        with tf.control_dependencies(
            [
                self.vx1[: tf.shape(metx)[0]].assign(metx),
                self.vx2[: tf.shape(metx)[0]].assign(metx),
            ]
        ):
            vx, vy, vz = neutrino_reco(
                lepe,
                lepx,
                lepy,
                lepz,
                metx,
                mety,
                self.vx1,
                self.vx2,
                self.opt1,
                self.opt2,
                self.mass_constraint,
            )
        return vx, vy, vz

We can now use our Neutrino reconstruction layer to build a `tf.keras` model. This model can be evaluated locally or loaded onto a tensorflow-model-server. It can additionally be saved to disc and is thus very portable.

In [ ]:
inp = tf.keras.layers.Input(shape=(6,))
reco = NeutrinoReco(
    mass_constraint=80.4,
    optimizer="Adam",
    batch_size=100,
)
vx, vy, vz = reco(inp)
outp = [
    tf.keras.layers.Lambda(lambda x: x, name=n)(t)
    for (n, t) in [("x", vx), ("y", vy), ("z", vz)]
]
model = tf.keras.models.Model(inputs=inp, outputs=outp)

We will now define an exemplary input event:

In [ ]:
# define input data in the structure [[lepe, lepx, lepy, lepz, metx, mety]]
events = np.array([[ 35.  , -34.   ,  -1.,  -6. , -63.  , -72.  ]])

Now we can evaluate it, using the `tf.keras` model we just have built:

In [ ]:
%%time
nu_x, nu_y, nu_z = model.predict(events, batch_size=100)
print(f"Neutrino: px={nu_x[0]:.2f}, px: {nu_y[0]:.2f}, px: {nu_z[0]:.2f}")

We have seen that the calculations through the graph have been performed succesfully. However, the execution took quite so time. This time is attributed to the intitial setup of the graph structure.

As it has been setup once now, all following executions are much faster: 

In [ ]:
%%time
nu_x, nu_y, nu_z = model.predict(events, batch_size=100)
print(f"Neutrino: px={nu_x[0]:.2f}, px: {nu_y[0]}, px: {nu_z[0]:.2f}")

Great, looks like it is pretty fast right now!

This concludes our tutorial for a graph based neutrino reconstruction. Congrats for making it through!

Should you have further questions, please contact me at: dennis.noll \[at\] rwth-aachen.de